<a href="https://colab.research.google.com/github/lusala-louis/IoT/blob/main/IoT_library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import Adafruit_DHT
import MFRC522
import RPi.GPIO as GPIO
import time
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Initialize Firebase
cred = credentials.Certificate("path/to/serviceAccountKey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://firebase-project.firebaseio.com/'
})

# Firebase Realtime Database reference
ref = db.reference('/sensor_data')

# GPIO setup for DHT11 temperature and humidity sensor
dht_pin = 4
dht_sensor = Adafruit_DHT.DHT11

# GPIO setup for RFID sensor (MFRC522)
rfid_pin_rst = 22
rfid_pin_ss = 24

# Initialize the RFID sensor
MIFAREReader = MFRC522.MFRC522()

# Function to read temperature and humidity from DHT11 sensor
def read_temperature_humidity():
    humidity, temperature = Adafruit_DHT.read_retry(dht_sensor, dht_pin)
    if humidity is not None and temperature is not None:
        print(f"Temperature: {temperature}°C")
        print(f"Humidity: {humidity}%")

        # Send data to Firebase
        data = {
            'temperature': temperature,
            'humidity': humidity
        }
        ref.child('temperature_humidity').set(data)
    else:
        print("Failed to retrieve data from DHT11 sensor.")

# Function to read RFID tag data from MFRC522 sensor
def read_rfid():
    (status, TagType) = MIFAREReader.MFRC522_Request(MIFAREReader.PICC_REQIDL)

    if status == MIFAREReader.MI_OK:
        print("RFID tag detected.")

        # Get the UID of the tag
        (status, uid) = MIFAREReader.MFRC522_Anticoll()

        if status == MIFAREReader.MI_OK:
            uid_str = ":".join([str(i) for i in uid])
            print(f"RFID tag UID: {uid_str}")

            # Send data to Firebase
            data = {
                'rfid_uid': uid_str
            }
            ref.child('rfid_data').set(data)
        else:
            print("Failed to read RFID tag.")

# Main loop
try:
    while True:
        # Read temperature and humidity
        read_temperature_humidity()

        # Read RFID tag
        read_rfid()

        # Delay between readings
        time.sleep(1)

except KeyboardInterrupt:
    GPIO.cleanup()
